# A Guided Tour of Ray Core: Remote Objects

© 2019-2022, Anyscale. All Rights Reserved

### Learning objectives

In this tutorial, you learn about:
 * Ray Futures as one of the patterns
 * Ray's distributed Plasma object stores
 * How obejcts are stored and fetched from the distributed shared object store
 * Use `ray.get` and `ray.put` examples


[*Remote Objects*](https://docs.ray.io/en/latest/walkthrough.html#objects-in-ray)
implement a [*shared-memory object store*](https://en.wikipedia.org/wiki/Shared_memory) pattern.

Objects are immutable and can be accessed from anywhere on the cluster, as they are stored in the cluster shared memory. An object ref is essentially a pointer or a unique ID that can be used to refer to a remote object without seeing its value. If you’re familiar with futures, Ray object refs are conceptually similar.


In general, small objects are stored in their owner’s **in-process store** (**<=100KB**) while large objects are stored in the **distributed object store**. This decision is meant to reduce the memory footprint and resolution time for each object. Note that in the latter case, a placeholder object is stored in the in-process store to indicate the object has been promoted to shared memory.

In the case if there is no space in the shared-memory, objects are spilled over to disk. But the main point here is that
shared-memory allows _zero-copy_ to processes on the same worker node.

<img src="images/shared_memory_plasma_store.png"  height="40%" width="65%">

---

## 2. Object references as Futures Pattern

First, let's start Ray…

In [71]:
import logging
import numpy as np
from pprint import pprint
import ray

if ray.is_initialized:
    ray.shutdown()
context = ray.init(logging_level=logging.ERROR)
pprint(context)

RayContext(dashboard_url='127.0.0.1:8269', python_version='3.8.13', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-23_11-44-58_818909_80673/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-23_11-44-58_818909_80673/sockets/raylet', 'webui_url': '127.0.0.1:8269', 'session_dir': '/tmp/ray/session_2022-07-23_11-44-58_818909_80673', 'metrics_export_port': 60580, 'gcs_address': '127.0.0.1:61498', 'address': '127.0.0.1:61498', 'node_id': 'da5a1427fa850326497f0be32d8233edaf46b205381f35f4c26764a0'})


In [72]:
print(f"Dashboard url: http://{context.address_info['webui_url']}")

Dashboard url: http://127.0.0.1:8269


## Remote Objects example

To start, we'll create some python objects and put them in shared memory

In [73]:
%%time
num_list = [23, 42, 93]

# returns an objectRef
obj_ref = ray.put(num_list)
obj_ref

CPU times: user 505 µs, sys: 448 µs, total: 953 µs
Wall time: 696 µs


ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000014000000)

Then retrieve the value of this object reference. This follows an object resolution protocol.

Small objects are resolved by copying them directly from the _owner’s_ **in-process store**. For example, if the owner calls `ray.get`, the system looks up and deserializes the value from the local **in-process store**. For larger objects greater than 100KB, they will be stored in the distributed object store.

In [74]:
val = ray.get(obj_ref)
val

[23, 42, 93]

You can gather the values of multiple object references in parallel using comprehension:
 1. Each value is put in the object store and its `ObjRefID` is immediately returned
 2. The comprehension constructs a list of `ObjRefIDs` for each element in the loop
 3. A final `get(list_obj_refs`) is invoked to fetch the list

In [75]:
results = ray.get([ray.put(i) for i in range(10)])
results

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### Passing Objects by Reference

Ray object references can be freely passed around a Ray application. This means that they can be passed as arguments to tasks, actor methods, and even stored in other objects. Objects are tracked via distributed reference counting, and their data is automatically freed once all references to the object are deleted.

In [76]:
# Define a Task
@ray.remote
def echo(x):
    print(f"current value of x: {x}")

In [77]:
x = list(range(10))
obj_ref_x = ray.put(x)
y = 25

### Pass-by-value

Send the object to a task as a top-level argument.
The object will be *de-referenced* automatically, so the task only sees its value.

In [78]:
echo.remote(y)

ObjectRef(fa03f7e1dcd75626ffffffffffffffffffffffff0100000001000000)

In [79]:
echo.remote(obj_ref_x)

ObjectRef(0c2b1bfc63ea50deffffffffffffffffffffffff0100000001000000)

(echo pid=90349) current value of x: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
(echo pid=90345) current value of x: 25


### Pass-by-reference

When passed inside a Python list or other data structure,
the *object ref is preserved*. The object data is not transferred to the worker
when it is passed by reference, until `ray.get()` is called on the reference.

You can pass by reference in two ways:
 1. as a dictionary `.remote({"obj": obj_ref_x})`
 2. as list of objRefs `.remote([obj_ref_x])`

In [80]:
x = list(range(20))
obj_ref_x = ray.put(x)
echo.remote({"obj": obj_ref_x})

ObjectRef(3a7ff36aab7246e2ffffffffffffffffffffffff0100000001000000)

(echo pid=90349) current value of x: {'obj': ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000020000000)}


In [81]:
echo.remote([obj_ref_x])

ObjectRef(903a108ba4971bacffffffffffffffffffffffff0100000001000000)

(echo pid=90349) current value of x: [ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000020000000)]


## What about long running tasks?

Now let's set a timeout to return early from attempted access of a remote object that is blocking for too long...

In [82]:
import time

@ray.remote
def long_running_function ():
    time.sleep(10)
    return 42

You can control how long you want to wait for the task to finish

In [83]:
%%time

from ray.exceptions import GetTimeoutError

obj_ref = long_running_function.remote()

try:
    ray.get(obj_ref, timeout=6)
except GetTimeoutError:
    print("`get` timed out")

`get` timed out
CPU times: user 21.2 ms, sys: 15.4 ms, total: 36.6 ms
Wall time: 6.03 s


In [84]:
ray.shutdown()

### Exercises

1. Send a list of object references returned by `ray.put(x)` 
2. Use comprehension to construct this list and send it to `my_function.remote(list_of_object_refs)` to return the sum of the list
3. Create a python object, use `ray.put(pobj)` and `ray.get(pobj)`
4. Create large lists and python dictionaries, put them in object store.

### Homework

1. Read references to get advanced deep dives and more about Ray objects
2. [Serialization](https://docs.ray.io/en/latest/ray-core/objects/serialization.html)
3. [Memory Management](https://docs.ray.io/en/latest/ray-core/objects/memory-management.html)
4. [Object Spilling](https://docs.ray.io/en/latest/ray-core/objects/object-spilling.html)
5. [Fault Tolerance](https://docs.ray.io/en/latest/ray-core/objects/fault-tolerance.html)

### Next Step

We covered how to use Ray `tasks`, `ray.get()` and `ray.put`, understand distributed remote object store, let's move on to the [Ray Actors lesson](ex_03_remote_classes.ipynb).

## References

 * [Ray Architecture Reference](https://docs.google.com/document/d/1lAy0Owi-vPz2jEqBSaHNQcy2IBSDEHyXNOQZlGuj93c/preview#)
 * [Ray Internals: A peek at ray,get](https://www.youtube.com/watch?v=a1kNnQu6vGw)
 * [Ray Internals: Object management with Ownership Model](https://www.anyscale.com/events/2021/06/22/ray-internals-object-management-with-the-ownership-model)
 * [Deep Dive into Ray scheduling Policies](https://www.anyscale.com/events/2021/06/23/a-deep-dive-into-rays-scheduling-policy)
 * [Redis in Ray: Past and future](https://www.anyscale.com/blog/redis-in-ray-past-and-future)
 * [StackOverFlow: How Ray Shares Data](https://stackoverflow.com/questions/58082023/how-exactly-does-ray-share-data-to-workers/71500979#71500979)
 